In [1]:
import os as o
import joblib as jb
from tqdm import tqdm
from keras.applications import VGG16
from keras.preprocessing.image import load_img, img_to_array
import cv2
import numpy as np
from keras.applications.vgg16 import preprocess_input
import numpy as np
from mtcnn import MTCNN
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image


In [2]:
characters=o.listdir('cropped_faces')
characters

['Hinata Hyuga',
 'Megumi Fushiguro',
 'Mikasa Ackerman',
 'Monkey D. Luffy',
 'Nami',
 'Naruto Uzumaki',
 'Nico Robin',
 'Roronoa Zoro',
 'Saitama',
 'Sakura Haruno',
 'Sanji',
 'Tsunade',
 'Yuji Itadori',
 'Zenitsu Agatsuma']

In [3]:
all_img = []
for i in characters:
    for j in o.listdir(o.path.join('cropped_faces', i)):
        all_img.append(o.path.join('cropped_faces', i, j))


In [4]:
len(all_img)

1928

In [5]:
jb.dump(all_img,open('all_imag.jbl','wb'))

In [6]:
face_model=jb.load('all_imag.jbl','rb')

In [7]:
model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='avg')

In [8]:
print(model.summary())

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

None


In [9]:
def image_det(img_path, model):
    try:     
        if not o.path.exists(img_path):
            print(f"File does not exist: {img_path}")
            return None
        img = cv2.imread(img_path)
        if img is None:
            print(f"Error loading image {img_path}: Image is None")
            return None        
        img = cv2.resize(img, (224, 224))        
        array_of_image = np.array(img, dtype=np.float32)        
        expand_img = np.expand_dims(array_of_image, axis=0)    
        preproces_img = preprocess_input(expand_img)
        result = model.predict(preproces_img).flatten()
        return result
    except Exception as e:
        print(f"General error processing image {img_path}: {e}")
        return None

In [10]:
attribute = []

for file in tqdm(face_model):
    result = image_det(file, model)
    if result is not None:
        attribute.append(result)


  0%|          | 0/1928 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step


  0%|          | 1/1928 [00:08<4:22:06,  8.16s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 371ms/step


  0%|          | 2/1928 [00:08<1:57:04,  3.65s/it]

...

100%|█████████▉| 1926/1928 [15:18<00:00,  2.21it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 434ms/step


100%|█████████▉| 1927/1928 [15:19<00:00,  2.03it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 442ms/step


100%|██████████| 1928/1928 [15:20<00:00,  2.10it/s]


In [11]:
print(len(attribute))
jb.dump(attribute,('attribute.jbl'))

1928


['attribute.jbl']

In [175]:
attribute_list=np.array(jb.load(open("attribute.jbl",'rb')))

In [287]:
detector=MTCNN()
test_img=cv2.imread("shail.jpg")

In [288]:
detect_face=detector.detect_faces(test_img)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step


In [289]:
x,y,width,height=detect_face[0]['box']

In [290]:
face=test_img[y:y+height,x:x+width]

In [291]:
image=Image.fromarray(face)
image=image.resize((224,224))

In [292]:
face_array=np.asarray(image)
face_array=face_array.astype('float32')

In [293]:
expanded_image=np.expand_dims(face_array,axis=0)
preprocessed_image=preprocess_input(expanded_image)

In [294]:
result=model.predict(preprocessed_image).flatten()
result

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 362ms/step


array([5.37632331e-02, 8.46307874e-02, 0.00000000e+00, 3.20271945e+00,
       3.54552686e-01, 0.00000000e+00, 5.02948388e-02, 9.60960209e-01,
       1.86841321e+00, 9.52330947e-01, 3.94387698e+00, 3.23935241e-01,
       3.98502946e+00, 0.00000000e+00, 0.00000000e+00, 2.74650425e-01,
       5.01239955e-01, 5.66901565e-01, 0.00000000e+00, 3.26534271e-01,
       2.01385260e-01, 4.66667175e-01, 7.53610730e-01, 4.61490583e+00,
       0.00000000e+00, 0.00000000e+00, 1.72273600e+00, 0.00000000e+00,
       1.58582163e+00, 0.00000000e+00, 4.11402918e-02, 3.57900381e-01,
       6.66310644e+00, 2.17463985e-01, 6.76796734e-01, 1.93942773e+00,
       1.62265015e+00, 2.83137858e-01, 1.68920651e-01, 0.00000000e+00,
       7.33538866e-01, 7.26577461e-01, 3.17488074e-01, 0.00000000e+00,
       1.60276055e-01, 0.00000000e+00, 2.69285083e-01, 4.22565985e+00,
       0.00000000e+00, 1.87133163e-01, 0.00000000e+00, 1.06294872e-02,
       5.99463940e-01, 0.00000000e+00, 1.53127193e+00, 1.33447671e+00,
      

In [295]:
result.shape

(512,)

In [296]:
cosine_similarity(result.reshape(1,-1),attribute_list[0].reshape(1,-1))[0][0]

0.31449968

In [297]:
simliarity=[]
for i in range (len(attribute_list)):
    sim_score=cosine_similarity(result.reshape(1,-1),attribute_list[i].reshape(1,-1))[0][0]
    simliarity.append(sim_score)

image_position=sorted(list(enumerate(simliarity)),reverse=True,key=lambda x: x[1] )[0][0]




In [298]:
temp_img=cv2.imread(all_img[image_position])
cv2.imshow('output',temp_img)
cv2.waitKey(0)

-1